# Preprocess Data

- This notebook will show on a small scale the functions of preprocess.py. 
- Here, we will download a few panoramic images, reorient and reproject them, making them ready for sidewalk extraction.

In [ ]:
import numpy as np

import sys

import json
import os
import requests
import shutil

import pandas as pd
from PIL import Image

### 0. Download Panoramic Images (DataPunt)

In [ ]:
# Make a call to the API to get the panoramas in the area of interest after 2021-01-01

# Base API url (subset: 10 images from page 150)
base_url = "https://api.data.amsterdam.nl/panorama/panoramas/?format=json&page_size=10&page=150&srid=4326&bbox={},{},{},{}&timestamp_after=2021-01-01" 

# Bounding box coordinates for the area of interest (Osdorp)
bounds = (4.754844, 52.346254, 4.820438, 52.381408)

# Create the url
url = base_url.format(*bounds)

test_api = requests.get(url).json()

In [ ]:
# The structure of the API json is as follows:
# - links: href used to get the json, next page, previous page
# - count: number of panoramas returned by the API
# - _embedded: the actual json with the panoramas
# -- panoramas: list of panoramas
# Each panorama has the following keys: 
# '_links', 'cubic_img_baseurl', 'cubic_img_pattern', 'geometry', 'pano_id', 'timestamp', 
# 'filename', 'surface_type', 'mission_distance', 'mission_type', 'mission_year', 'tags', 'roll', 'pitch', 'heading'
# The link to get our panorama is in '_links' -> 'equirectangular_full' -> 'href'

# Print number of panoramas in the area of interest
print("Number of panoramas in the area of interest: {}".format(test_api['count']))


# Collect all the links to the panoramas and the panorama ids
pano_ids = [pano['pano_id'] for pano in test_api['_embedded']['panoramas']]
links = [pano['_links']['equirectangular_full']['href'] for pano in test_api['_embedded']['panoramas']]
#print(test_api['_embedded']['panoramas'][0].keys())
#print(test_api['_embedded']['panoramas'][0])
print(pano_ids)
print(links)

In [ ]:
# Make a folder to store the panoramas called sample_dataset
folder = '../res/sample_dataset/'
os.makedirs(folder, exist_ok=True)

success = 0
fail = 0
for link in links:
    # The filename of the panorama is the corresponding pano_id value
    filename = pano_ids[links.index(link)]
    # Download the panorama
    res = requests.get(link, allow_redirects=True)
    # Save the panorama
    if res.status_code == 200:
        with open(folder + filename, 'wb') as f:
            f.write(res.content)
            success += 1
    else:
        fail += 1
print(f"Success: {success}, Fail: {fail}")

In [ ]:
# Save the panos with their headings to a csv
headings = [pano['heading'] for pano in test_api['_embedded']['panoramas']]
data = {'pano_id': pano_ids, 'heading': headings}
# The headers of the csv are: pano_id, heading
df = pd.DataFrame(data)
df.to_csv(folder + 'panos.csv', index=False)

### 1. Reorient panoramas

In [ ]:
# Reshift panorama according to heading
def orient_panorama(img, heading):

    shift = heading/360
    pixel_split = int(img.size[0] * shift)
    
    left = Image.fromarray(np.array(img)[:, pixel_split:])
    right = Image.fromarray(np.array(img)[:, :pixel_split])
    
    reoriented_img = np.hstack((left, right))
    
    return Image.fromarray(reoriented_img)

In [ ]:
# Define path to .csv 
csvpath = folder + 'panos.csv'
csv = pd.read_csv(csvpath)

# Make a folder to store the reoriented panoramas called sample_dataset_reoriented
folder_reoriented = folder + 'reoriented/'
os.makedirs(folder_reoriented, exist_ok=True)

# Reorient all the images in the .csv file and save them in a new folder
for index, row in csv.iterrows():
    img_filename = row['pano_id']
    
    img = Image.open(folder + img_filename, formats=['JPEG'])
    heading = row['heading']
    reoriented_img = orient_panorama(img, heading)
    reoriented_img.save(folder_reoriented + img_filename, format='JPEG')

### 2. Reproject panoramas

Convert from Equirectangular projection to Cubemaps projection

In [ ]:
import vrProjector as vrProjector
from vrProjector.EquirectangularProjection import EquirectangularProjection
from vrProjector.CubemapProjection import CubemapProjection

In [ ]:
# We manually changed the mode from 'RGBA' to 'RGB' in AbstractProjection.py since we are dealing with .jpg files
def split(img, pano_id, size):

    # VrProjector the images to cubemap
    eq = EquirectangularProjection()
    eq.loadImage(img)
    cb = CubemapProjection()
    cb.initImages(size,size)
    cb.reprojectToThis(eq)

    # Retrieve front, right, back, left images
    front = Image.fromarray(np.uint8(cb.front))
    right = Image.fromarray(np.uint8(cb.right))
    back = Image.fromarray(np.uint8(cb.back))
    left = Image.fromarray(np.uint8(cb.left))

    # Print mode of front
    print(front.mode)

    # Make a directory and save images in it
    directory = os.path.join(folder + 'reprojected/', pano_id)
    if not os.path.exists(directory):
            os.makedirs(directory)

    front.save(os.path.join(directory, 'front.png'))
    right.save(os.path.join(directory, 'right.png'))
    back.save(os.path.join(directory, 'back.png'))
    left.save(os.path.join(directory, 'left.png'))

    print('saved {}!'.format(pano_id))

In [ ]:
# Load sample image
pano_id = os.listdir(folder + 'reoriented/')[0]
img = os.path.join(folder + 'reoriented', pano_id)
size = 512

split(img, pano_id, size)

In [ ]:
# Show right image
right_path = os.path.join(folder + 'reprojected/', pano_id, 'right.png')
right = Image.open(right_path)
right.show()